In [1]:
import pickle
import datetime
import numpy as np
import pandas as pd
from keras import layers
from keras.models import Sequential
from keras.utils import plot_model
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
LOCAL_PATH = 'D:\\waiting time\\我的代码\\TrainData\\'
# 加载数据
def pickleLoad(filename, local_path = LOCAL_PATH):
    with open(local_path + filename, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        data = u.load()
    return data


#def pickleDump(data_to_pickle, filename, local_path = LOCAL_PATH):
 #   """ Function to write data to pickle format. """
  #  pickle.dump(data_to_pickle, open(local_path + filename, "wb+"))


In [3]:
# 将字符串转换成日期
def StrToDate(date_str):
    date = datetime.datetime.strptime(date_str,'%Y-%m-%d').date()
    return date

In [4]:
# 比较两次相邻的duration所间隔的天数
def compareDate(date1, date2, ndays = 7):
    interval = (date2 - date1).days
    if interval >= ndays:
        return True
    else:
        return False

In [5]:
# 加载之前保存的旧数据和新数据，然后合并两部分数据
print ("Load Old and New data...")
local_path1 = 'D:\\waiting time\\我的代码\\'
raw_treatments_new = pickleLoad('raw_treatments1.p', local_path1)
local_path2 = 'D:\\waiting time\\data\\Regression\\all_data\\'
raw_treatments_old = pickleLoad('PreInputData.p', local_path2)

Load Old and New data...


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\waiting time\\我的代码\\raw_treatments1.p'

In [ ]:
raw_treatments_new_dict = {}
for pat in raw_treatments_new.keys():
    appt_dict = {}
    for cou in raw_treatments_new[pat].keys():
        for fra in raw_treatments_new[pat][cou].keys():
            for date in raw_treatments_new[pat][cou][fra].keys():
                appt_new = raw_treatments_new[pat][cou][fra][date]
                appt_new['course'] = cou
                appt_new['fraction'] = fra
                appt_dict[date] = appt_new
    raw_treatments_new_dict[pat] = appt_dict

In [ ]:
raw_treatments_old_dict = {}
for pat in raw_treatments_old.keys():
    appt_dict = {}
    for cou in raw_treatments_old[pat].keys():
        for fra in raw_treatments_old[pat][cou].keys():
            for date in raw_treatments_old[pat][cou][fra].keys():
                appt_new = raw_treatments_old[pat][cou][fra][date]
                appt_new['course'] = cou
                appt_new['fraction'] = fra
                appt_dict[date] = appt_new
    raw_treatments_old_dict[pat] = appt_dict

In [ ]:
for pat in raw_treatments_old_dict.keys():
    if pat in raw_treatments_new_dict.keys():
        for date in raw_treatments_old_dict[pat].keys():
            if date in raw_treatments_new_dict[pat].keys():
                pass
            else:
                raw_treatments_new_dict[pat][date] = raw_treatments_old_dict[pat][date]
                
    else:
        raw_treatments_new_dict[pat] = raw_treatments_old_dict[pat]


In [ ]:
print ("Save raw_treatments_new_dict")
pickleDump(raw_treatments_new_dict, 'raw_treatments_new_dict.p')

print ("\nLoad raw_treatments_new_dict")
raw_treatments_new_dict = pickleLoad('raw_treatments_new_dict.p')


In [ ]:
# patient_ser, course, fraction, date
# 没有用的特征 'radiation_ser', 'prev_duration', 'avg_duration', 'allocated'
# 不确定的 tot_mu, tot_mucoeff
# 数量太多，不使用 tot_mu, tot_mucoeff, plan, radiation_id
feature_nonnum = ['course',
                  'fraction',
                  'therapist',
                  'resource',
                  'diagnosis',
                  'oncologist',
                  'appt_day_of_week',
                  'appt_month',
                  'appt_hour',
                  'orientation',
                  'gender',]

feature_num = ['num_of_fields',
               'images_taken',
               'image_duration',
               'age',]

feature_duration = ['duration']



In [ ]:
# 数据标准化
def getAveAndStd(raw_treatments_new_dict, name):
    feature = []
    for pat in raw_treatments_new_dict.keys():
        for date in raw_treatments_new_dict[pat].keys():
            text = raw_treatments_new_dict[pat][date]
            feature.append(text[name])
    
    feature = np.array(feature)
    feature_ave = np.mean(feature)
    feature_std = np.std(feature)
    
    return feature_ave, feature_std

In [ ]:
# 得到每个特征对应的平均值和标准差
def GetFeatureAveStdDict(raw_treatments_new_dict, feature_num = feature_num):
    feature_ave_std_dict = {}
    for name in feature_num:
        print('\n\n', name)
        feature_ave, feature_std = getAveAndStd(raw_treatments_new_dict, name)
        feature_ave_std_dict[name + '_ave'] = feature_ave
        feature_ave_std_dict[name + '_std'] = feature_std
    return feature_ave_std_dict

feature_ave_std_dict_new = GetFeatureAveStdDict(raw_treatments_new_dict)

In [ ]:
# 获得数据的one-hot字典，同时进行数据归一化
def GetFeatureDictAndNorm(raw_treatments_new_dict, feature_ave_std_dict_new):
    print('\nGet Nonnum One-hot Dictionary and Num normalize...')
    print ("\nLoad feature_onehot_dict")
    feature_onehot_dict = pickleLoad('feature_onehot_dict.p')
    
    raw_treatments_new_list = []
    pat_ser_set = []
    pat_ser_list = []
    date_list = []
    for pat in raw_treatments_new_dict.keys():
        pat_ser_set.append(pat)
        for date in sorted(raw_treatments_new_dict[pat]):
            pat_ser_list.append(pat)
            date_list.append(StrToDate(date))
            treat_list = [pat, StrToDate(date)]
            text = raw_treatments_new_dict[pat][date]
            
            # 数值
            for name in feature_num:
                value = text[name]
                value = value - feature_ave_std_dict_new[name + '_ave']
                value = value / feature_ave_std_dict_new[name + '_std']
                treat_list.append(value)
                
                # 非数值
            for name in feature_nonnum:
                treat_list.append(text[name])
            duration = sum(text['duration'])
            treat_list.append([duration])
            raw_treatments_new_list.append(treat_list)
    
    print('\nThe length of the raw_treatments_new_list: ', len(raw_treatments_new_list))        
    print('\nShow some samples:')
    for i in raw_treatments_new_list[:2]:
        print(i)
        print('\n')
        
    print('\nSave raw_treatments_new_list')
    pickleDump(raw_treatments_new_list, 'raw_treatments_new_list.p')    
    
    
    return feature_onehot_dict, raw_treatments_new_list, pat_ser_set, pat_ser_list


In [ ]:
feature_onehot_dict, raw_treatments_new_list, pat_ser_set, pat_ser_list = GetFeatureDictAndNorm(raw_treatments_new_dict, feature_ave_std_dict_new)


In [ ]:
print ("\nLoad raw_treatments_new_list")
raw_treatments_new_list = pickleLoad('raw_treatments_new_list.p')


In [ ]:
# 将不同患者，以及时间间隔过大的数据进行分割
def GetCutList(raw_treatments_new_list, pat_ser_list, pat_ser_set):
    raw_treatments_new_cut = []
    raw_treatments_new_cut_len = []
    #raw_len = []
    for i in range(len(pat_ser_set) - 1):
        start = pat_ser_list.index(pat_ser_set[i])
        end = pat_ser_list.index(pat_ser_set[i + 1])
        
        one_pat = raw_treatments_new_list[start: end]
#        one_pat.sort(key = lambda one_pat: one_pat[3])
        
        flag = 0
        for appt_i in range(len(one_pat) - 1):
            date1 = one_pat[appt_i][1]
            date2 = one_pat[appt_i + 1][1]
            if compareDate(date1, date2):
                raw_treatments_new_cut.append(one_pat[flag: appt_i + 1])
                raw_treatments_new_cut_len.append(len(one_pat[flag: appt_i + 1]))
                flag = appt_i + 1
        if flag != len(one_pat):
            raw_treatments_new_cut.append(one_pat[flag: ])
            raw_treatments_new_cut_len.append(len(one_pat[flag: appt_i + 1]))
    
    max_idx = raw_treatments_new_cut_len.index(max(raw_treatments_new_cut_len))
    pat_treatment = raw_treatments_new_cut[max_idx]
    
    print('\nSave raw_treatments_new_cut')
    pickleDump(raw_treatments_new_cut, 'raw_treatments_new_cut.p')
    
    return raw_treatments_new_cut, pat_treatment


In [ ]:
raw_treatments_new_cut, pat_treatment = GetCutList(raw_treatments_new_list, pat_ser_list, pat_ser_set)
TIME_STEPS = len(pat_treatment)
print ("\nLoad raw_treatments_new_cut")
raw_treatments_new_cut = pickleLoad('raw_treatments_new_cut.p')


In [ ]:
# 把第一次治疗的数据提出来
# 之前使用了第一次的duration当作sample_sequence的一个特征
# 现在区分第一次的duration，sample_sequence从第三次治疗开始
def SepFirst(raw_treatments_new_cut):
    feature = len(raw_treatments_new_cut[0][0])
    sample_first_new = []
    sample_sequence_new = []
    
    
    print('Get sample_first and sample_sequence')
    for i in range(len(raw_treatments_new_cut)):
        pat = raw_treatments_new_cut[i]
        if len(pat) == 1:
            pass
        else:
            if pat[0][7] == 1:
                sample_first_new.append(pat[0])
                prev_duration = []
                for i in range(len(pat) - 1):
                    prev_duration.append(pat[i][-1])
                
                pat_sequence = pat[1:]
                for i in range(len(pat_sequence)):
                    pat_sequence[i].append(prev_duration[i])
                sample_sequence_new.append(pat_sequence)
                
            else:
                sample_first_new.append([0] * feature)
                prev_duration = [[0]]
                for i in range(len(pat) - 1):
                    prev_duration.append(pat[i][-1])
                
                pat_sequence = pat
                for i in range(len(pat_sequence)):
                    pat_sequence[i].append(prev_duration[i])
                sample_sequence_new.append(pat_sequence)
    
    print('\nSave sample_first_new')
    pickleDump(sample_first_new, 'sample_first_new.p')
    print('\nSave sample_sequence_new')
    pickleDump(sample_sequence_new, 'sample_sequence_new.p')
    
    return sample_first_new, sample_sequence_new

sample_first_new, sample_sequence_new = SepFirst(raw_treatments_new_cut)


In [ ]:
# 第二次的治疗，prev_duration是第一次的治疗时间
# 因此，将第二次的治疗和sequence的治疗分开
def CutSecondAppt(sample_first_new, sample_sequence_new):
    count_ind = []
    sample_first_new_cut = []
    sample_second = []
    sample_sequence_new_cut = []
    
    for i in range(len(sample_sequence_new)):
        pat_first = sample_first_new[i]
        pat_sequence = sample_sequence_new[i]
        if len(pat_sequence) > 1:
            sample_first_new_cut.append(pat_first)
            pat_second = pat_sequence[0]
            sample_second.append(pat_second)
            pat_cut = pat_sequence[1:]
            sample_sequence_new_cut.append(pat_cut)
            count_ind.append(i)
    
    return count_ind, sample_first_new_cut, sample_second, sample_sequence_new_cut

_, sample_first_new_cut, sample_second, sample_sequence_new_cut = CutSecondAppt(sample_first_new, sample_sequence_new)



In [ ]:
def get_prev_bench_mark(sample_sequence_new_cut):
    cost_list = []
    
    for i in range(len(sample_sequence_new_cut)):
        # 得到一个病人 patient
        pat_sequence = sample_sequence_new_cut[i]
        
        for j in range(len(pat_sequence)):
            appt = pat_sequence[j]
            prev_duration = appt[-2]
            now_duration = appt[-1]
            cost = abs(prev_duration[0] - now_duration[0])
            cost_list.append(cost)
    
    bench_mark = sum(cost_list) / len(cost_list)    
    return bench_mark, cost_list

bench_mark, _ = get_prev_bench_mark(sample_sequence_new_cut)
# 5.9724657627689925

In [ ]:
def Onehot(term, onehot_dict):
    result = np.zeros((len(onehot_dict), ))
    
    if type(term) == list:
        for i in term:
            if i != 0:
                result[onehot_dict.index(i)] = 1
    else:
        if term != 0:
            result[onehot_dict.index(term)] = 1
    
    return result

In [ ]:
def CombineOnehot(appt, FEATURE, F_onehot = feature_onehot_dict):
    result = np.zeros((FEATURE, ))
    
    if FEATURE == 746:
        if type(appt[5]) == int:
            result[:5] = appt[2:6] + appt[-1]
        else:
            appt_num = appt[2:5] + appt[5].tolist() + appt[-1]
            result[:5] = appt_num
        start_ind = 5
    else:
        if type(appt[5]) == int:
            result[:4] = appt[2:6]
        else:
            appt_num = appt[2:5] + appt[5].tolist()
            result[:4] = appt_num
        start_ind = 4
        
    appt_nonnum = appt[6:17]
    for i in range(len(appt_nonnum)):
        name = feature_nonnum[i] + '_onehot'
        onehot_dict = F_onehot[name]
        result[start_ind: start_ind + len(onehot_dict)] = Onehot(appt_nonnum[i], onehot_dict)
        start_ind = start_ind + len(onehot_dict)
        
    return result

In [ ]:
# 使用one-hot编码，根据最长的特征向量，其他的特征向量填补零
# 将所有数据补成等长的
def GetPadding(pat, j, TIME_STEPS, FEATURE):
    one_sample = np.zeros((TIME_STEPS, FEATURE))
    
    for n in range(len(pat[: j + 1])):
        appt = pat[n]
        re = CombineOnehot(appt, FEATURE)
        one_sample[n, :] = re
    
    return one_sample

In [ ]:
def CategoricalFeature(appt):
    pass

In [ ]:
def get_batch_dense(train_sample, count_pre, BATCH, FEATURE):
    count = 0
    n = 0
    
    batch_train = np.zeros((BATCH, FEATURE))
    batch_y = np.zeros((BATCH, ))
    
    for i in range(len(train_sample)):
        # onehot
        if count < count_pre:
            count = count + 1
        elif count >= count_pre and (count - count_pre) < BATCH:
            # 得到appt
            appt = train_sample[i]
            batch_train[n, :] = CombineOnehot(appt, FEATURE)
            batch_y[n] = appt[-1][0]
            n = n + 1
            count = count + 1
        else:
            break
#    print('count is ', count)
    state = np.random.get_state()    
    np.random.shuffle(batch_train)
    np.random.set_state(state)
    np.random.shuffle(batch_y)
    
    return batch_train, batch_y, count

In [ ]:
def get_batch_sequence(train_sequence_new, count_pre, BATCH, TIME_STEPS, FEATURE):    
    count = 0
    n = 0
    
    batch_train_sequence = np.zeros((BATCH, TIME_STEPS, FEATURE))
    batch_y_sequence = np.zeros((BATCH, ))
    
    for i in range(len(train_sequence_new)):
        # 得到一个病人 patient
        pat_sequence = train_sequence_new[i]
        
        for j in range(len(pat_sequence)):
            appt = pat_sequence[j]
            
            if count < count_pre:
                count = count + 1
            elif count >= count_pre and (count - count_pre) < BATCH:
                batch_y_sequence[n] = appt[-2][0]
                one_sample = GetPadding(pat_sequence, j, TIME_STEPS, FEATURE)
                batch_train_sequence[n, :, :] = one_sample
                n = n + 1
                count = count + 1
            else:
                break
    
#    print('count is ', count)
    state = np.random.get_state()    
    np.random.shuffle(batch_train_sequence)
    np.random.set_state(state)
    np.random.shuffle(batch_y_sequence)
    
    return batch_train_sequence, batch_y_sequence, count


In [ ]:
def Dense_model(FEATURE):
    model_Dense = Sequential()
    model_Dense.add(layers.Dense(
            activation='relu',
            batch_input_shape=(None, FEATURE),
            output_dim = 128,
            ))
    
    model_Dense.add(layers.Dropout(0.5))
    
    model_Dense.add(layers.Dense(
            activation='relu',
            output_dim = 32,
            ))
    
    model_Dense.add(layers.Dropout(0.5))
    
    model_Dense.add(layers.Dense(
            activation='relu',
            output_dim = 1,
            ))

    model_Dense.summary()
    model_Dense.compile(optimizer='rmsprop', loss='mae')
    
    return model_Dense

In [ ]:
def GRU_model(TIME_STEPS, FEATURE):
    # 后续治疗，fraction number > 1
    model_sequence = Sequential()
    model_sequence.add(layers.GRU(
            batch_input_shape = (None , TIME_STEPS, FEATURE),
            output_dim = 128,
            dropout=0.1,
            recurrent_dropout=0.5,
            return_sequences=True,
            ))
    
    model_sequence.add(layers.GRU(
            output_dim = 32,
            dropout=0.1,
            recurrent_dropout=0.5,
            ))
    # stateful = True 本次batch的参数返回到下一次的训练中
    
    model_sequence.add(layers.Dense(1))
    
    model_sequence.compile(
            optimizer = 'rmsprop',
            loss = 'mae'
            )
    
    model_sequence.summary()
    return model_sequence

In [ ]:
def plotTrainingCost(STEP_PLOT, COST_PLOT, BATCH, STEPS, plot_step, NAME):
    plt.plot(STEP_PLOT, COST_PLOT, color='blue', label='training loss')
    plt.plot(STEP_PLOT, [4.6] * len(COST_PLOT), color='red', label='Random Forest loss')
    plt.legend(loc = 'upper right')
    plt.ylabel('Train Cost (%d Steps-%d Batch)'%(plot_step, BATCH))
    plt.xlabel('Steps')
    plt.title('training_cost_%s-%dtime_steps- %dbatch'%(NAME, STEPS, BATCH))
    plt.grid()
    plt.rcParams['savefig.dpi'] = 300  # 图片像素
    plt.rcParams['figure.dpi'] = 300 # 分辨率
    plt.rcParams['figure.figsize'] = (15,15)  # 尺寸
    plt.savefig(LOCAL_PATH + "training_cost_%s-%dtime_steps-%dbatch.png"%(NAME, STEPS, BATCH), format='png')
    plt.show()

In [ ]:
def plotRealPredict(result_sequence, batch_y_test_sequence, BATCH, BATCH_TEST, STEPS, plot_step, NAME):
    plt.plot(batch_y_test_sequence, color='red', label='real')
    plt.plot(result_sequence, color='blue', label='predict')
    plt.legend(loc = 'upper right')
    plt.ylabel('Train Cost (%d Steps-%d Batch)/minutes'%(plot_step, BATCH_TEST))
    plt.xlabel('Appointments')
    plt.title('Real-Prediction_%s-%dSteps-%dbatch'%(NAME, STEPS, BATCH))
    plt.grid()
    plt.rcParams['savefig.dpi'] = 300  # 图片像素
    plt.rcParams['figure.dpi'] = 300 # 分辨率
    plt.rcParams['figure.figsize'] = (15,15)  # 尺寸
    plt.savefig(LOCAL_PATH + "Real-Prediction_%s-%dtime_steps-%dbatch.png"%(NAME, STEPS, BATCH), format='png')
    plt.show()

In [ ]:
train_sample = raw_treatments_new_list[:230000]
test_sample = raw_treatments_new_list[230000:]
print('\ntrain_samplet length is ', len(train_sample))
print('\ntest_sample length is ', len(test_sample))

In [ ]:
BATCH_DENSE = 10
FEATURE_DENSE = 745
print('\nTraining ------------')
COUNT_PRE_DENSE = 0
COST_DENSE = 0
COST_PLOT_DENSE = []
STEP_PLOT_DENSE = []
plot_step_dense = 100
print('\nbatch_train shape is : (%d, %d)'%(BATCH_DENSE, FEATURE_DENSE))
print('\nbatch_y shape is : (%d, )'%BATCH_DENSE)
STEPS_DENSE = len(train_sample) // BATCH_DENSE
print(STEPS_DENSE)

model_dense = Dense_model(FEATURE_DENSE)
STEPS_DENSE = 23000
for step in range(STEPS_DENSE):
    #    print('COUNT_PRE_first:', COUNT_PRE_first)
    batch_train, batch_y, COUNT_PRE_DENSE = get_batch_dense(train_sample, COUNT_PRE_DENSE, BATCH_DENSE, FEATURE_DENSE)
    cost_Dense = model_dense.train_on_batch(batch_train, batch_y)
    
    if (step + 1) % plot_step_dense == 0:
#        print('\n%d round average train cost is %f'%((step + 1), cost_Dense))
        print('\n%d round average train cost is %f'%((step + 1), (COST_DENSE / plot_step_dense)))
        COST_PLOT_DENSE.append(COST_DENSE / plot_step_dense)
        STEP_PLOT_DENSE.append(step + 1)
        COST_DENSE = 0
    else:
        COST_DENSE = COST_DENSE + cost_Dense

In [ ]:
COUNT_PRE_DENSE_TEST = 0
BATCH_DENSE_TEST = 200
results_dense = []
targets_dense = []
for step in range(1):
    print('Test %d round ', (step + 1))
    
    batch_train_dense, batch_y_dense, COUNT_PRE_DENSE_TEST = get_batch_dense(train_sample, COUNT_PRE_DENSE_TEST, BATCH_DENSE_TEST, FEATURE_DENSE)
    result_dense = model_dense.predict_on_batch(batch_train_dense)
#    targets_dense.append(batch_y_dense)
#    results_dense.append(result_dense)

In [ ]:
NAME = 'all_dense'
plotTrainingCost(STEP_PLOT_DENSE, COST_PLOT_DENSE, BATCH_DENSE, STEPS_DENSE, plot_step_dense, NAME)
plotRealPredict(result_dense, batch_y_dense, BATCH_DENSE, BATCH_DENSE_TEST, STEPS_DENSE, plot_step_dense, NAME)

In [ ]:
model_dense = Dense_model(FEATURE_DENSE)
plot_model(model_dense, to_file = LOCAL_PATH + 'model_dense.png', show_shapes = True)


In [ ]:
#train_first_new = sample_first_new[:15000]
#test_first_new = sample_first_new[15000:]
#
#train_sequence_new = sample_sequence_new[:15000]
#test_sequence_new = sample_sequence_new[15000:]

train_first_new_cut = sample_first_new_cut[:15000]
test_first_new_cut = sample_first_new_cut[15000:]

train_sequence_new_cut = sample_sequence_new_cut[:15000]
test_sequence_new_cut = sample_sequence_new_cut[15000:]

def CountSampleInSequence(train_sequence_new_cut):
    total_sample_number = 0
    for pat in train_sequence_new_cut:
        for appt in pat:
            total_sample_number = total_sample_number + 1
    return total_sample_number

total_sample_number = CountSampleInSequence(train_sequence_new)
# 239992
total_sample_number_cut = CountSampleInSequence(train_sequence_new_cut)
# 229862


In [ ]:
BATCH = 100
STEPS = total_sample_number // BATCH
STEPS_cut = total_sample_number_cut // BATCH
print(STEPS_cut)

print('\nTraining ------------')
COUNT_PRE_sequence = 0
COST_sequence = 0
TIME_STEPS = 50
FEATURE = 746
plot_step = 50
COST_PLOT = []
STEP_PLOT = []
model_sequence = GRU_model(TIME_STEPS, FEATURE)
print('\nbatch_train_first shape is : (%d, %d)'%(BATCH, FEATURE))
print('\nbatch_train_first shape is : (%d, )'%BATCH)
print('\nbatch_train_sequence shape is : (%d, %d, %d)'%(BATCH, TIME_STEPS, FEATURE))
print('\nbatch_train_sequence shape is : (%d, )'%BATCH)
#STEPS = len(raw_treatments_new_list) // BATCH
#STEPS = 2350
STEPS = 2250
#check_batch_y_sequence = []
for step in range(STEPS):
    batch_train_sequence, batch_y_sequence, COUNT_PRE_sequence = get_batch_sequence(train_sequence_new_cut, COUNT_PRE_sequence, BATCH, TIME_STEPS, FEATURE)
#    check_batch_y_sequence.append(batch_y_sequence)
    cost_sequence = model_sequence.train_on_batch(batch_train_sequence, batch_y_sequence)
    
    if (step + 1) % plot_step == 0:
        print('%d round average train cost is %f'%((step + 1), (COST_sequence / plot_step)))
        COST_PLOT.append(COST_sequence / plot_step)
        STEP_PLOT.append(step + 1)
        COST_sequence = 0
    else:
        COST_sequence = COST_sequence + cost_sequence


In [ ]:
model_sequence = GRU_model(TIME_STEPS, FEATURE)
plot_model(model_sequence, to_file = LOCAL_PATH + 'model_sequence.png', show_shapes = True)


In [ ]:
#print ("Save raw_treatments_new_dict")
#pickleDump(raw_treatments_new_dict, 'raw_treatments_new_dict.p')
#print ("Save raw_treatments_new_dict")
#pickleDump(raw_treatments_new_dict, 'raw_treatments_new_dict.p')
#print ("Save raw_treatments_new_dict")
#pickleDump(raw_treatments_new_dict, 'raw_treatments_new_dict.p')


#model_sequence.save('model_sequence_cut_second_dropout_%dbatch_%dsteps_NoCut.h5'%(BATCH, STEPS))

#from keras.models import load_model
#model_sequence = load_model('model_sequence_cut_second_%dbatch_%dsteps_NoCut.h5'%(BATCH, STEPS))


In [ ]:
def count_better(COST_PLOT):
    count_better = []
    for i in COST_PLOT:
        if i < 4.6:
            count_better.append(i)
    return count_better

count_better = count_better(COST_PLOT)


In [ ]:
COUNT_PRE = 0
BATCH_TEST = 200
results_sequence = []
targets_sequence = []
for step in range(1):
    print('Test %d round ', (step + 1))
    
    batch_test_sequence, batch_y_test_sequence, COUNT_PRE = get_batch_sequence(test_sequence_new_cut, COUNT_PRE, BATCH_TEST, TIME_STEPS, FEATURE)
    result_sequence = model_sequence.predict(batch_test_sequence)
#    results_sequence.append(result_sequence)
#    targets_sequence.append(batch_y_test_sequence)

NAME = 'cut_dropout'
plotTrainingCost(STEP_PLOT, COST_PLOT, BATCH, STEPS, plot_step, NAME)
plotRealPredict(result_sequence, batch_y_test_sequence, BATCH, BATCH_TEST, STEPS, TIME_STEPS, plot_step, NAME)
